# Finale Aufgabe für Praktikum Deep Learning <br>Textgenerierung mit RNN: Modelltraining

* **Name:** Fabian Schotte
* **Email:** fabian.schotte@rwu.de
* **Matrikelnummer:** 35604
* **Studiengang:** Angewandte Informatik

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import matplotlib.pylab as plt
from work import models
import time 

os.makedirs("models", exist_ok=True)

2025-06-24 20:56:53.369558: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-24 20:56:53.376902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750798613.385529     560 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750798613.388140     560 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-24 20:56:53.396909: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Vorbereitung


### Laden der Trainingsdaten
Hier werden die Trainings- und Testdaten der Kaggle Sentiment Analyis aus deren CSV-Dateien ausgelesen und die Inhalte der Spalte `text` zu dem String `kaggle_text` zusammengefasst.

In [2]:
df_train = pd.read_csv('work/kaggle_sentiment/tweet_sentiment_train.csv', encoding='utf-8', encoding_errors='replace')
df_test = pd.read_csv('work/kaggle_sentiment/tweet_sentiment_test.csv', encoding='utf-8', encoding_errors='replace')

kaggle_text_train = df_train['text'].str.cat(sep='\n')
kaggle_text_test = df_test['text'].str.cat(sep='\n')
# kaggle_text = kaggle_text_train + '\n' + kaggle_text_test
kaggle_text = kaggle_text_train
# kaggle_text = kaggle_text_test

print(kaggle_text[:500])

 I`d have responded, if I were going
 Sooo SAD I will miss you here in San Diego!!!
my boss is bullying me...
 what interview! leave me alone
 Sons of ****, why couldn`t they put them on the releases we already bought
http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
2am feedings for the baby are fun when he is all smiles and coos
Soooo high
 Both of you
 Journey!? Wow... u just became cooler.  hehe... (is that possible!?)
 as much as i love to be hopef


Im nächsten Codeblock wird ein Set der einzigartigen Charaktere im String `kaggle_text` mit dem Namen `vocab` erstellt. Ebenso werden die darin vorhandenen Charaktere ausgegeben und die Länge des Sets ausgegeben.

In [3]:
vocab = sorted(set(kaggle_text))
print(vocab)
print(f"vocab size = {len(vocab)}")

['\t', '\n', ' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '´', '½', '¿', 'Â', 'ï']
vocab size = 102


## Preprocessing
Im folgenden Codeblock wird ein Beispieltext zu einer Liste von Charakteren aufgeteilt und ausgegeben. Diese List wird als `chars` gespeichert.

In [4]:
example_texts = ['hello world', 'hello world']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

I0000 00:00:1750798615.429478     560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9397 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


<tf.RaggedTensor [[b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd'],
 [b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd']]>

In der Funktion `ids_from_chars` werden die gegebenen Charaktere in eine zugeordnete Zahl für das Training und die Vorhersage umgewandelt.
Die Funktion `chars_from_ids` funktioniert genau umgekehrt, indem hier die gegebenen Zahlen in die dazugehörigen Charaktere umgewandelt werden.
Hier werden auch die beiden Methoden für den Beispieltext aus `chars` durchgeführt und ausgegeben. Zur Vollständigkeit werden die Charaktere, die aus den Zahlenwerten generiert wurden, wieder zu einem String zusammengefügt.

In [5]:
ids_from_chars = keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
print(ids)

chars_from_ids = keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
print(chars)

tf.strings.reduce_join(chars, axis=-1).numpy()

<tf.RaggedTensor [[74, 71, 78, 78, 81, 3, 89, 81, 84, 78, 70],
 [74, 71, 78, 78, 81, 3, 89, 81, 84, 78, 70]]>
<tf.RaggedTensor [[b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd'],
 [b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd']]>


array([b'hello world', b'hello world'], dtype=object)

In `text_from_ids(ids)` wird genau die Operation für das Zusammenfügen der Charaktere zu einem String ausgeführt.

In [6]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Hier wird die zuvor genannte Funktion zur Berechnung der Zahlenwerte aus den Charakteren `kaggle_text` verwendet, um die Trainingsdaten als Zahlenwerte zu erhalten. Diese werden in `all_ids` gespeichert.

In [7]:
all_ids = ids_from_chars(tf.strings.unicode_split(kaggle_text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1905188,), dtype=int64, numpy=array([ 3, 43, 66, ..., 85, 11, 11])>

Als nächstes wird ein Dataset aus den Zahlenwerten generiert. Die ersten 10 Werte des Datasets werden als Charaktere ausgegeben.

In [8]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

 
I
`
d
 
h
a
v
e
 


2025-06-24 20:56:56.329995: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Mit `seq_lenth` wird die Länge der Sequenz definiert, mit dem die Modelle trainiert werden.

In [9]:
seq_length = 100

Im nächsten Schritt werden die Sequenzen mit Hilfe von `seq_lenth` generiert. Die Daten für die Sequenzeb kommen aus dem zuvor angelegten Dataset `ids_dataset`.
Dazu werden auch die Charaktere der ersten Sequenz zuerst einzeln und dann als Text ausgegeben.

In [10]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq).numpy())
for seq in sequences.take(1):
  print(text_from_ids(seq).numpy())

[b' ' b'I' b'`' b'd' b' ' b'h' b'a' b'v' b'e' b' ' b'r' b'e' b's' b'p'
 b'o' b'n' b'd' b'e' b'd' b',' b' ' b'i' b'f' b' ' b'I' b' ' b'w' b'e'
 b'r' b'e' b' ' b'g' b'o' b'i' b'n' b'g' b'\n' b' ' b'S' b'o' b'o' b'o'
 b' ' b'S' b'A' b'D' b' ' b'I' b' ' b'w' b'i' b'l' b'l' b' ' b'm' b'i'
 b's' b's' b' ' b'y' b'o' b'u' b' ' b'h' b'e' b'r' b'e' b' ' b'i' b'n'
 b' ' b'S' b'a' b'n' b' ' b'D' b'i' b'e' b'g' b'o' b'!' b'!' b'!' b'\n'
 b'm' b'y' b' ' b'b' b'o' b's' b's' b' ' b'i' b's' b' ' b'b' b'u' b'l'
 b'l' b'y' b'i']
b' I`d have responded, if I were going\n Sooo SAD I will miss you here in San Diego!!!\nmy boss is bullyi'


2025-06-24 20:56:56.426928: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In `split_input_target(sequence)` werden die Inputs und Target Labels der Sequenz generiert

In [11]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

Im folgenden Code wird die zuvor erstellte Methode `split_input_target` auf die Sequenzen der Kaggle Trainingsdaten angewendet und die Länge des Datasets ausgegeben

In [12]:
kaggle_dataset = sequences.map(split_input_target)
len(kaggle_dataset)

18863

Als nächstes werden Beispiele für die Input und Labels als Text und mit dessen Shape ausgegeben.

In [ ]:
for input_example, target_example in kaggle_dataset.take(1):
    print("Input:", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())
    print("Input shape:", input_example.shape)
    print("Target shape:", target_example.shape)

Input: b' I`d have responded, if I were going\n Sooo SAD I will miss you here in San Diego!!!\nmy boss is bully'
Target: b'I`d have responded, if I were going\n Sooo SAD I will miss you here in San Diego!!!\nmy boss is bullyi'
Input shape: (100,)
Target shape: (100,)


2025-06-24 20:56:56.603954: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Als nächstes werden folgende Variablen für das Training definiert:
- Mit `BATCH_SIZE` wird definiert, wie viele Input-Label-Paare in einer Epoche verarbeitet werden. Hier sind dies 150
- Mit `BUFFER_SIZE` wird die Anzahl der Elemente des Datasets definiert, die zufällig gemischt werden, bevor sie in die Batches eingeteilt werden.

Dieses Variablen werden hier auch auf das Dataset `kaggle_dataset` angewendet und dieses Dataset wird ebenfalls ausgegeben. 

In [14]:
BATCH_SIZE = 150
BUFFER_SIZE = 1000

kaggle_dataset = (
    kaggle_dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)
kaggle_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(150, 100), dtype=tf.int64, name=None), TensorSpec(shape=(150, 100), dtype=tf.int64, name=None))>

### 1. GRU-Modell

In [16]:
vocab_size_1 = len(ids_from_chars.get_vocabulary())
embedding_dim_1 = 256
rnn_units_1 = 2048

In [17]:
inputs_1 = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_1 = keras.layers.Embedding(input_dim=vocab_size_1, output_dim=embedding_dim_1)(inputs_1)
gru_1, gru_state_1 = keras.layers.GRU(units=rnn_units_1, return_sequences=True, return_state=True)(embedding_1)
gru_1, gru_state_1 = keras.layers.GRU(units=rnn_units_1, return_sequences=True, return_state=True)(gru_1)
outputs_1 = keras.layers.Dense(units=vocab_size_1, activation='softmax')(gru_1)

gru_model_1 =  keras.Model(inputs=inputs_1, outputs=outputs_1)

#### Testen des Modells

In [18]:
for input_example_batch, target_example_batch in kaggle_dataset.take(1):
    example_batch_predictions_model_1 = gru_model_1(input_example_batch)
    print(example_batch_predictions_model_1.shape, "# (batch_size, sequence_length, vocab_size_1)")

I0000 00:00:1750798623.455405     698 cuda_dnn.cc:529] Loaded cuDNN version 90501


(150, 100, 103) # (batch_size, sequence_length, vocab_size)


In [19]:
gru_model_1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_tokens (InputLayer)       │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │        26,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ [(None, None, 2048),   │    14,168,064 │
│                                 │ (None, 2048)]          │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ [(None, None, 2048),   │    25,178,112 │
│                                 │ (None, 2048)]          │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 103)      │       211,047 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,583,591 (151.00 MB)

 Trainable params: 39,583,591 (151.00 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
sampled_indices_model_1 = tf.random.categorical(example_batch_predictions_model_1[0], num_samples=1)
sampled_indices_model_1 = tf.squeeze(sampled_indices_model_1, axis=-1).numpy()
sampled_indices_model_1

array([ 99,  68,  57,  59,  32,  66,  12, 102,  87,  90,  43,  45,  86,
        33,  52,  70,  20,  85,  51,  55,  25,  45,  79,   3,  59,  24,
        30,  82,  52,  48,  15,  80,  36,  17,  69,   8,  29,  51,  80,
        48,  82,  50,  40,  88,  57,  68,  63,  63,   0,  45,  60,  77,
        67,  58,  40,  51,  50,  24,  53,  89,   7,  77,  33,  18,  50,
        23,  92,   6,  10,  31,  63,  34,  90,  82,  40,  86,  82,  61,
        96,  18,  70,  97,  40,   8,  18,  44,  28,  19,  26,  25,  81,
         3,  79,  64,  85,  36,  25,  19,  55,  21])

In [21]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_model_1).numpy())

Input:
 b'd Claudia.  Dianne is new to Tweetville, Claudia has been on a while but not active here\nMy legs are'

Next Char Predictions:
 b'\xc2\xbdbWY>`*\xc3\xafuxIKt?Rd2sQU7Km Y6<pRN-nB/c&;QnNpPFvWb]][UNK]KZkaXFQP6Sw%k?0P5z$(=]@xpFtp[~0d\xc2\xa0F&0J:187o m^sB71U3'


### Training

#### Loss

In [22]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [23]:
example_batch_mean_loss_model_1 = loss(target_example_batch, example_batch_predictions_model_1)
print("Prediction shape: ", example_batch_predictions_model_1.shape, " # (batch_size, sequence_length, vocab_size_1)")
print("Mean loss:        ", example_batch_mean_loss_model_1)

Prediction shape:  (150, 100, 103)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.634601, shape=(), dtype=float32)


/opt/conda/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


In [24]:
tf.exp(example_batch_mean_loss_model_1).numpy()

np.float32(102.98683)

#### Optimizer

In [25]:
gru_model_1.compile(optimizer='adam', loss=loss, metrics=['accuracy'], run_eagerly=True)

#### Early Stopping

In [26]:
early_stopping_gru_model_1 = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)

#### Konfiguration von Checkpoints

In [27]:
checkpoint_dir = './work/training_checkpoints/gru_model_1'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_1=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

In [28]:
EPOCHS = 30

In [29]:
start = time.perf_counter()
gru_model_1_history = gru_model_1.fit(kaggle_dataset, epochs=EPOCHS, callbacks=[checkpoint_callback_gru_model_1, early_stopping_gru_model_1])
end = time.perf_counter()
gru_model_1_training_time = end - start

Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 47s 368ms/step - accuracy: 0.1808 - loss: 3.6047
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 368ms/step - accuracy: 0.3915 - loss: 2.2056
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 358ms/step - accuracy: 0.4879 - loss: 1.8279
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 361ms/step - accuracy: 0.5286 - loss: 1.6605
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 368ms/step - accuracy: 0.5510 - loss: 1.5661
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 363ms/step - accuracy: 0.5701 - loss: 1.4870
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 359ms/step - accuracy: 0.5878 - loss: 1.4152
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 365ms/step - accuracy: 0.6068 - loss: 1.3384
Epoch 9/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 370ms/step - accuracy: 0.6288 - loss: 1.2546
Epoch 10/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 366ms/step - accuracy: 0.6547 - loss: 1.1589
Epoch 11/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 363ms/step - accuracy: 0.6867 - loss: 1.0496
Epoch 12/30
125/125

#### Speichern

In [ ]:
gru_model_1.save('work/models/gru_model_1.keras')

### 2. GRU-Modell

In [ ]:
vocab_size_2 = len(ids_from_chars.get_vocabulary())
embedding_dim_2 = 256
rnn_units_2 = 1024

In [ ]:
inputs_2 = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_2 = keras.layers.Embedding(input_dim=vocab_size_2, output_dim=embedding_dim_2)(inputs_2)
gru_2, gru_state_2 = keras.layers.GRU(units=rnn_units_2, return_sequences=True, return_state=True)(embedding_2)
dropout_2 = keras.layers.Dropout(0.2)(gru_2)
gru_2, gru_state_2 = keras.layers.GRU(units=rnn_units_2, return_sequences=True, return_state=True)(dropout_2)
outputs_2 = keras.layers.Dense(units=vocab_size_2, activation='softmax')(gru_2)

gru_model_2 = keras.Model(inputs=inputs_2, outputs=outputs_2)

#### Testen des Modells

In [ ]:
for input_example_batch, target_example_batch in kaggle_dataset.take(1):
    example_batch_predictions_gru_model_2 = gru_model_2(input_example_batch)
    print(example_batch_predictions_gru_model_2.shape, "# (batch_size, sequence_length, vocab_size_2)")

In [ ]:
gru_model_2.summary()

In [ ]:
sampled_indices_gru_model_2 = tf.random.categorical(example_batch_predictions_gru_model_2[0], num_samples=1)
sampled_indices_gru_model_2 = tf.squeeze(sampled_indices_gru_model_2, axis=-1).numpy()
sampled_indices_gru_model_2

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_gru_model_2).numpy())

### Training

#### Loss

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss_model_2 = loss(target_example_batch, example_batch_predictions_gru_model_2)
print("Prediction shape: ", example_batch_predictions_gru_model_2.shape, " # (batch_size, sequence_length, vocab_size_2)")
print("Mean loss:        ", example_batch_mean_loss_model_2)

In [ ]:
tf.exp(example_batch_mean_loss_model_1).numpy()

#### Optimizer

In [ ]:
gru_model_2.compile(optimizer='adam', loss=loss, metrics=['accuracy'], run_eagerly=True)

#### Early Stopping

In [ ]:
early_stopping_model_2 = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)

#### Konfiguration von Checkpoints

In [ ]:
checkpoint_dir = './work/training_checkpoints/gru_model_2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_2=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

In [ ]:
EPOCHS = 30

In [ ]:
start = time.perf_counter()
gru_model_2_history = gru_model_2.fit(kaggle_dataset, epochs=EPOCHS, callbacks=[checkpoint_callback_gru_model_2, early_stopping_model_2])
end = time.perf_counter()
gru_model_2_training_time = end - start

### Speichern

In [ ]:
gru_model_2.save('work/models/gru_model_2.keras')

## Bewertung und Vergleich der GRU-Modelle

In [ ]:
loss1 = gru_model_1_history.history['loss']
loss2 = gru_model_2_history.history['loss']

accuracy1 = gru_model_1_history.history['accuracy']
accuracy2 = gru_model_2_history.history['accuracy']

perplexity1 = np.exp(loss1)
perplexity2 = np.exp(loss2)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4)) 

axes[0].plot(loss1, label="GRU Model 1", linestyle='-', color='blue')
axes[0].plot(loss2, label="GRU Model 2", linestyle='-', color='red')
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Loss")
axes[0].set_title("Comparison of Training Histories")
axes[0].legend()

axes[1].plot(accuracy1, label="GRU Model 1", linestyle='-', color='blue')
axes[1].plot(accuracy2, label="GRU Model 2", linestyle='-', color='red')
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Accuracy Comparison")
axes[1].legend()

axes[2].plot(perplexity1, label="GRU Model 1", linestyle='-', color='blue')
axes[2].plot(perplexity2, label="GRU Model 2", linestyle='-', color='red')
axes[2].set_xlabel("Epochs")
axes[2].set_ylabel("Perplexity")
axes[2].set_title("Perplexity Comparison")
axes[2].legend()

plt.tight_layout()

plt.show()

print("Loss:")
print(f" GRU Model 1: {loss1[-1]}")
print(f" GRU Model 2: {loss2[-1]}")
print("Accuracy:")
print(f" GRU Model 1: {accuracy1[-1]}")
print(f" GRU Model 2: {accuracy2[-1]}")
print("Perplexity:")
print(f" GRU Model 1: {perplexity1[-1]}")
print(f" GRU Model 2: {perplexity2[-1]}")
print("Training Times:")
print(f" GRU Model 1: {gru_model_1_training_time:2f}s")
print(f" GRU Model 2: {gru_model_2_training_time:2f}s")


### LSTM-Modell

In [ ]:
vocab_size_lstm = len(ids_from_chars.get_vocabulary())
embedding_dim_lstm = 256
rnn_units_lstm = 2048

In [ ]:

inputs_lstm = keras.layers.Input(shape=(None,), dtype='int32', name='input_tokens')
embedding_lstm = keras.layers.Embedding(input_dim=vocab_size_lstm, output_dim=embedding_dim_lstm)(inputs_lstm)
lstm, hidden_state_1, cell_state_1 = keras.layers.LSTM(units=rnn_units_lstm, return_sequences=True, return_state=True)(embedding_lstm)
lstm, hidden_state_2, cell_state_2 = keras.layers.LSTM(units=rnn_units_lstm, return_sequences=True, return_state=True)(lstm)
outputs_lstm = keras.layers.Dense(units=vocab_size, activation='softmax')(lstm)

lstm_model = keras.Model(inputs=inputs_lstm, outputs=outputs_lstm)

#### Testen des Modells

In [ ]:
for input_example_batch, target_example_batch in kaggle_dataset.take(1):
    example_batch_predictions_lstm_model = lstm_model(input_example_batch)
    print(example_batch_predictions_lstm_model.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
lstm_model.summary()

In [ ]:
sampled_indices_lstm_model = tf.random.categorical(example_batch_predictions_lstm_model[0], num_samples=1)
sampled_indices_lstm_model = tf.squeeze(sampled_indices_lstm_model, axis=-1).numpy()
sampled_indices_lstm_model

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices_gru_model_2).numpy())

### Training

#### Loss

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss_lstm_model = loss(target_example_batch, example_batch_predictions_lstm_model)
print("Prediction shape: ", example_batch_predictions_lstm_model.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss_lstm_model)

In [ ]:
tf.exp(example_batch_mean_loss_lstm_model).numpy()

#### Optimizer

In [ ]:
lstm_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'], run_eagerly=True)

#### Konfiguration von Checkpoints

In [ ]:
checkpoint_dir = './work/training_checkpoints/lstm_model'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback_gru_model_2=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Ausführen des Trainings

In [ ]:
EPOCHS = 30

In [ ]:
start = time.perf_counter()
lstm_model_history = lstm_model.fit(kaggle_dataset, epochs=EPOCHS, callbacks=[checkpoint_callback_gru_model_2, early_stopping_model_2])
end = time.perf_counter()
lstm_model_training_time = end - start

#### Speichern

In [ ]:
lstm_model.save('work/models/lstm_model.keras')

## Vergleich aller Modelle

In [ ]:
loss1 = gru_model_1_history.history['loss']
loss2 = gru_model_2_history.history['loss']
loss3 = lstm_model_history.history['loss']

accuracy1 = gru_model_1_history.history['accuracy']
accuracy2 = gru_model_2_history.history['accuracy']
accuracy3 = lstm_model_history.history['accuracy']

perplexity1 = np.exp(loss1)
perplexity2 = np.exp(loss2)
perplexity3 = np.exp(loss3)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4)) 

axes[0].plot(loss1, label="GRU Model 1", linestyle='-', color='blue')
axes[0].plot(loss2, label="GRU Model 2", linestyle='-', color='red')
axes[0].plot(loss3, label="LSTM Model", linestyle='-', color='green')
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Loss")
axes[0].set_title("Comparison of Training Histories")
axes[0].legend()

axes[1].plot(accuracy1, label="GRU Model 1", linestyle='-', color='blue')
axes[1].plot(accuracy2, label="GRU Model 2", linestyle='-', color='red')
axes[1].plot(accuracy3, label="LSTM Model", linestyle='-', color='green')
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Accuracy Comparison")
axes[1].legend()

axes[2].plot(perplexity1, label="GRU Model 1", linestyle='-', color='blue')
axes[2].plot(perplexity2, label="GRU Model 2", linestyle='-', color='red')
axes[2].plot(perplexity3, label="LSTM Model", linestyle='-', color='green')
axes[2].set_xlabel("Epochs")
axes[2].set_ylabel("Perplexity")
axes[2].set_title("Perplexity Comparison")
axes[2].legend()

plt.tight_layout()

plt.show()
print("Loss:")
print(f" GRU Model 1: {loss1[-1]}")
print(f" GRU Model 2: {loss2[-1]}")
print(f" LSTM Model:  {loss3[-1]}")
print("Accuracy:")
print(f" GRU Model 1: {accuracy1[-1]}")
print(f" GRU Model 2: {accuracy2[-1]}")
print(f" LSTM Model:  {accuracy3[-1]}")
print("Perplexity:")
print(f" GRU Model 1: {perplexity1[-1]}")
print(f" GRU Model 2: {perplexity2[-1]}")
print(f" LSTM Model:  {perplexity3[-1]}")
print("Training Times:")
print(f" GRU Model 1: {gru_model_1_training_time:2f}s")
print(f" GRU Model 2: {gru_model_2_training_time:2f}s")
print(f" LSTM Model:  {lstm_model_training_time:2f}s")